# Plotter Notebook

This notebook is used to plot the obtained results from the experiments in more sophisticated plots. If you want to use this notebook, you will need to name specific file names accordingly, but this notebook is not needed to see the results normally. If you want to reproduce the results, its sufficient to use the explorer notebook.

In [ ]:
# Imports
import os
import json
import numpy as np
import matplotlib.pyplot as plt
import scripts.framework as fr
from scipy.stats import zscore

In [ ]:
# Create representations
repr_sp = fr.Representation("./data/nodes_sp.csv", "./data/edges_sp.csv", maptype="sp")
repr_nl = fr.Representation("./data/nodes_nl.csv", "./data/edges_nl.csv", maptype="nl")

In [ ]:
def plot_results(dir, map_name, results_headrpp, results_nsga2, results_dijkstra):
    """
    Plots the results of the algorithms in sophisticated graphs.

    Args:
    - dir: the directory of the results
    - map_name: the name of the map
    - results_headrpp: the results of the HEADRPP algorithm
    - results_nsga2: the results of the NSGA-II algorithm
    - results_dijkstra: the results of Dijkstra's algorithm
    """

    fig, axs = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle(f"Comparison of HEADRPP and NSGA-II on a map of {map_name}")
    objectives = [
        (i + 1, obj)
        for i, obj in enumerate(["Distance", "Traffic", "Pollution", "Hotspots"])
    ]
    x_axis = 0
    y_axis = 0

    # Plot the objectives against each other
    for i, (index1, obj1) in enumerate(objectives):
        for j, (index2, obj2) in enumerate(objectives):
            if i >= j:
                continue
            ax = axs[x_axis, y_axis]
            y_axis += 1
            if y_axis == 3:
                x_axis += 1
                y_axis = 0
            ax.set_title(f"{obj1} vs {obj2}")
            ax.set_xlabel(obj1)
            ax.set_ylabel(obj2)

            # Extract the values of the objectives from the results
            obj1_values_headrpp = [result[index1] for result in results_headrpp]
            obj2_values_headrpp = [result[index2] for result in results_headrpp]
            obj1_values_nsga2 = [result[index1] for result in results_nsga2]
            obj2_values_nsga2 = [result[index2] for result in results_nsga2]
            obj1_values_dijk = [result[index1] for result in results_dijkstra]
            obj2_values_dijk = [result[index2] for result in results_dijkstra]

            ax.scatter(
                obj1_values_headrpp,
                obj2_values_headrpp,
                label="HEADRPP",
                color="blue",
                alpha=0.5,
                marker="D",
            )
            ax.scatter(
                obj1_values_nsga2,
                obj2_values_nsga2,
                label="NSGA-II",
                color="red",
                alpha=0.5,
                marker="D",
            )
            ax.legend()

    plt.tight_layout()
    plt.savefig(
        f"./results_replotted/{dir}_obj_{map_name.replace(' ', '_').lower()}.png"
    )
    plt.show()
    plt.close()

    # Plot the fitness against the objectives
    fig, axs = plt.subplots(2, 2, figsize=(10, 10))
    fig.suptitle(f"Fitness against objectives on a map of {map_name}")
    for i, (index, obj) in enumerate(objectives):
        ax = axs[i // 2, i % 2]
        ax.set_title(f"Fitness vs {obj}")
        ax.set_xlabel("Fitness")
        ax.set_ylabel(obj)

        obj_values_headrpp = [result[index] for result in results_headrpp]
        obj_values_nsga2 = [result[index] for result in results_nsga2]
        obj_values_dijk = [result[index] for result in results_dijkstra]
        fitness_values_headrpp = [result[0] for result in results_headrpp]
        fitness_values_nsga2 = [result[0] for result in results_nsga2]
        fitness_values_dijk = [result[0] for result in results_dijkstra]

        ax.scatter(
            fitness_values_headrpp,
            obj_values_headrpp,
            label="HEADRPP",
            color="blue",
            alpha=0.5,
            marker="D",
        )
        ax.scatter(
            fitness_values_nsga2, 
            obj_values_nsga2, 
            label="NSGA-II", 
            color="red",
            alpha=0.5,
            marker="D",
        )
        ax.legend()

    plt.tight_layout()
    plt.savefig(
        f"./results_replotted/{dir}_fit_{map_name.replace(' ', '_').lower()}.png"
    )
    plt.show()
    plt.close()


    time_headrpp = [result[-1] for result in results_headrpp]
    time_nsga2 = [result[-1] for result in results_nsga2]
    zscore_headrpp = zscore(time_headrpp)
    zscore_nsga2 = zscore(time_nsga2)
    time_headrpp = [time for i, time in enumerate(time_headrpp) if abs(zscore_headrpp[i]) < 3 and abs(zscore_nsga2[i]) < 3]
    time_nsga2 = [time for i, time in enumerate(time_nsga2) if abs(zscore_headrpp[i]) < 3 and abs(zscore_nsga2[i]) < 3]

    # Plot the time taken to run the algorithms
    fig, ax = plt.subplots()
    ax.set_yscale('log')
    log_ticks = [0.1, 0.2, 0.5, 1, 2, 5, 10, 20, 50, 100, 200, 500]
    ax.set_yticks(log_ticks)
    ax.get_yaxis().set_major_formatter(plt.ScalarFormatter())
    ax.boxplot(
        time_headrpp,
        positions=[1],
        labels=["HEADRPP"],
        patch_artist=True,
    )
    ax.boxplot(
        time_nsga2,
        positions=[2],
        labels=["NSGA-II"],
        patch_artist=True,
    )
    ax.set_title(f"Time taken to run the algorithms on a map of {map_name}")
    ax.set_ylabel("Time taken (s)")
    plt.savefig(
        f"./results_replotted/{dir}_time_{map_name.replace(' ', '_').lower()}.png"
    )
    plt.show()
    plt.close()

In [ ]:
# Define the routes and weights
routes = {1: {'sp' : (48, 433), 'nl' : (0, 142)},
          2: {'sp' : (103, 322), 'nl' : (88, 134)},
          3: {'sp' : (133, 334), 'nl' : (120, 83)}}
metrics = ['distance', 'traffic', 'pollution', 'hotspots']

# Loop over the directories and plot the results for the multi-objective experiments
dirs = os.listdir('./results_renamed')
for dir in dirs:
    if dir[3] == '_' and int(dir[4]) <= 3:
        if int(dir[4]) == 1:
            route = routes[int(dir.split('_')[-1][-1])]
            if int(dir.split('_')[-1][-1]) != 1:
                continue
            weights = [1/4, 1/4, 1/4, 1/4]
        elif int(dir[4]) == 2:
            route = routes[int(dir.split('_')[-2][-1])]
            weights = [1/6, 1/6, 1/6, 1/6]
            weights[metrics.index(dir.split('_')[-1])] = 1/2
        elif int(dir[4]) == 3:
            route = routes[int(dir.split('_')[-2][-1])]
            weights = [3/10, 3/10, 3/10, 3/10]
            weights[metrics.index(dir.split('_')[-1])] = 1/10
        else:
            raise ValueError("Invalid directory name")

        exp_data = json.loads(open(f"./results_renamed/{dir}/{dir}.json").read())
        results_headrpp_sp = exp_data['results_headrpp_sp']
        results_nsga2_sp = exp_data['results_nsga2_sp']
        results_headrpp_nl = exp_data['results_headrpp_nl']
        results_nsga2_nl = exp_data['results_nsga2_nl']

        plot_results(dir, 'Singapore', results_headrpp_sp, results_nsga2_sp, [])
        plot_results(dir, 'the Netherlands', results_headrpp_nl, results_nsga2_nl, [])

In [ ]:
# Plot the results of the algorithms in sophisticated graphs for the single-objective experiments
files = os.listdir('./results_renamed/exp_5_route2/results')
for file in files:

    route = None
    metric = None
    country = None
    country_proper = None
    
    route = 1
    metric = 'hotspots'
    country = 'netherlands'
    country_proper = 'the Netherlands'
    split = file.split('_')
    if split[0] == 'Route1':
        route = 1
        metric = split[2]
    elif split[0] == 'Route2':
        route = 2
        metric = split[2]
    elif split[3] == 'Route2':
        route = 1
        metric = split[1]
    elif split[3] == 'Route3':
        route = 3
        metric = split[1]
    else:
        raise ValueError("Could not determine route")

    if split[-1].split('.')[0] == 'Netherlands':
        country = 'netherlands'
        country_proper = 'the Netherlands'
    elif split[-1].split('.')[0] == 'Singapore':
        country = 'singapore'
        country_proper = 'Singapore'
    else:
        raise ValueError("Could not determine country")

    data = json.loads(open(f"./results_renamed/exp_5_route2/results/{file}").read())

    times = {'headrpp': data["results_headrpp"]["times"],
             'nsga2': data["results_nsga2"]["times"]
    }

    zscore_headrpp = zscore(data["results_headrpp"]["times"])
    zscore_nsga2 = zscore(data["results_nsga2"]["times"])
    times['headrpp'] = [time for i, time in enumerate(times['headrpp']) if abs(zscore_headrpp[i]) < 3 and abs(zscore_nsga2[i]) < 3]
    times['nsga2'] = [time for i, time in enumerate(times['nsga2']) if abs(zscore_headrpp[i]) < 3 and abs(zscore_nsga2[i]) < 3]

    # Plot the time taken to run the algorithms
    fig, ax = plt.subplots()
    ax.set_yscale('log')
    log_ticks = [0.1, 0.2, 0.5, 1, 2, 5, 10]
    ax.set_yticks(log_ticks)
    ax.yaxis.set_major_formatter(plt.FixedFormatter([str(l) for l in log_ticks]))
    ax.get_yaxis().set_major_formatter(plt.FixedFormatter([str(l) for l in log_ticks]))
    ax.boxplot(
        times['headrpp'],
        positions=[1],
        labels=["HEADRPP"],
        patch_artist=True,
    )
    ax.boxplot(
        times['nsga2'],
        positions=[2],
        labels=["NSGA-II"],
        patch_artist=True,
    )
    ax.set_yticks(log_ticks)
    ax.yaxis.set_major_formatter(plt.FixedFormatter([str(l) for l in log_ticks]))
    ax.get_yaxis().set_major_formatter(plt.FixedFormatter([str(l) for l in log_ticks]))
    ax.set_title(f"Time taken to run the algorithms on a map of {country_proper}")
    ax.set_ylabel("Time taken (s)")
    plt.savefig(
        f"./results_replotted/exp_5_route{route}_{metric}_time_{country}.png"
    )
    plt.show()
    plt.close()

    costs = {'headrpp': data["results_headrpp"]["costs"],
             'nsga2': data["results_nsga2"]["costs"],
             'dijkstra': data["results_dijkstra"]["costs"][1::2],
    }


    # Create histograms
    fig, axs = plt.subplots(3, 1, figsize=(8, 12))

    min_cost = min(min(costs['headrpp']), min(costs['nsga2']), min(costs['dijkstra']))
    max_cost = max(max(costs['headrpp']), max(costs['nsga2']), max(costs['dijkstra']))
    
    bin_width = (max_cost - min_cost) / 20
    bins = np.arange(min_cost, max_cost + bin_width, bin_width)

    # HEADRPP histogram
    axs[0].hist(costs['headrpp'], bins=bins, color='blue', edgecolor='black')
    axs[0].set_title('HEADRPP')
    axs[0].set_xlabel(metric.capitalize())
    axs[0].set_ylabel('Frequency')

    # NSGA-II histogram
    axs[1].hist(costs['nsga2'], bins=bins, color='green', edgecolor='black')
    axs[1].set_title('NSGA-II')
    axs[1].set_xlabel(metric.capitalize())
    axs[1].set_ylabel('Frequency')

    # Dijkstra histogram
    axs[2].hist(costs['dijkstra'], bins=bins, color='red', edgecolor='black')
    axs[2].set_title('Dijkstra')
    axs[2].set_xlabel(metric.capitalize())
    axs[2].set_ylabel('Frequency')

    for ax in axs:
        ax.set_xlim([min_cost - 0.1 * abs(max_cost - min_cost), max_cost + 0.1 * abs(max_cost - min_cost)])

    fig.suptitle(f"Costs for running the algorithms on a map of {country_proper} with '{metric}' objective")
    plt.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust layout to fit the main title
    plt.savefig(f"./results_replotted/exp_5_route{route}_{metric}_cost_{country}.png")
    plt.show()
    plt.close()